*This Jupyter Notebook goes through the entire process of building the Image Colorization model. \
First, we generate the training set using colored images, and then we define and train the model. \
Finally, we test our model by colorizing a black and white image.*

In [ ]:
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import img_to_array, load_img, array_to_img
from skimage.transform import resize
from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.layers import Input, RepeatVector, Reshape, Conv2D, UpSampling2D, concatenate
from keras.optimizers import Adam

from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

from keras.callbacks import ReduceLROnPlateau

import matplotlib.pyplot as plt

np.random.seed(42)

**I. Generate the training set**

Download inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5 from: https://www.kaggle.com/code/valkling/image-colorization-using-autoencoders-and-resnet/input

In [ ]:
inception = InceptionResNetV2(weights=None, include_top=True)
inception.load_weights('inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5')
inception.graph = tf.compat.v1.get_default_graph()

def get_inception_embedding(grey_rgb):
    grey_rgb_resized = resize(grey_rgb ,(1, 299, 299, 3))
    grey_rgb_resized = preprocess_input(grey_rgb_resized)
    embedding = inception.predict(grey_rgb_resized)
    return embedding[0]

def get_training_data(colored_imgs):
    l_imgs, ab_imgs, inception_embeddings= [], [], []
    for img in colored_imgs:
        gray_img= rgb2gray(img)
        gray_rgb_img= gray2rgb(gray_img)
        lab_img= rgb2lab(img)
        l_img = lab_img[:,:,:1]
        assert l_img.shape == (256,256,1), 'L channel matrix has a wrong shape'
        ab_img= lab_img[:,:,1:]/128.    # Standardize AB matrices

        l_imgs.append(l_img)
        ab_imgs.append(ab_img)
        inception_embeddings.append(get_inception_embedding(gray_rgb_img))

    l_imgs= np.stack(l_imgs, axis=0)  
    ab_imgs= np.stack(ab_imgs, axis=0)  
    inception_embeddings= np.stack(inception_embeddings, axis=0)
    return [l_imgs, inception_embeddings], ab_imgs

**Note:** Make sure you are refering to the correct path!

Download color file from: https://www.kaggle.com/code/theblackmamba31/autoencoder-grayscale-to-color-image/input

In [ ]:
# Images file path
path= '.'

#Standardize RGB images by dividing by 255
train_datagen = ImageDataGenerator(rescale=1./255)

#Resize images, if needed
train = train_datagen.flow_from_directory(path, classes=['color'], target_size=(256, 256), batch_size=7129, class_mode=None)

In [ ]:
n_train= 7100 # Number of Training samples

X_train, Y_train= get_training_data(train[0][:n_train])

**II. Colorization Model**

In [ ]:
def model():
    embed_input = Input(shape=(1000,))
    # Encoder
    encoder_input= Input(shape=(256, 256, 1))
    encoder_layer= Conv2D(64, (3, 3), activation='relu', padding='same', strides=2)(encoder_input)
    encoder_layer= Conv2D(128, (3, 3), activation='relu', padding='same')(encoder_layer)
    encoder_layer= Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_layer)
    encoder_layer= Conv2D(256, (3,3), activation='relu', padding='same')(encoder_layer)
    encoder_layer= Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_layer)
    encoder_layer= Conv2D(512, (3,3), activation='relu', padding='same')(encoder_layer)
    encoder_layer= Conv2D(512, (3,3), activation='relu', padding='same')(encoder_layer)
    encoder_output= Conv2D(256, (3,3), activation='relu', padding='same')(encoder_layer)

    # Fusion
    # Inception Embedding part
    fusion_layer = RepeatVector(32*32)(embed_input) 
    fusion_layer = Reshape(([32, 32, 1000]))(fusion_layer)
    # Adding the Encoder output
    fusion_layer = concatenate([encoder_output, fusion_layer], axis=3) 
    fusion_output = Conv2D(256, (1,1), activation='relu', padding='same')(fusion_layer)

    #Decoder
    decoder_layer= Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
    decoder_layer= UpSampling2D((2,2))(decoder_layer)
    decoder_layer= Conv2D(64, (3,3), activation='relu', padding='same')(decoder_layer)
    decoder_layer= UpSampling2D((2,2))(decoder_layer)
    decoder_layer= Conv2D(32, (3,3), activation='relu', padding='same')(decoder_layer)
    decoder_layer= Conv2D(16, (3,3), activation='relu', padding='same')(decoder_layer)
    decoder_layer= Conv2D(2, (3,3), activation='tanh', padding='same')(decoder_layer)
    decoder_output= UpSampling2D((2,2))(decoder_layer)

    return Model(inputs= [encoder_input, embed_input], outputs= decoder_output)

**III. Train the model**

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

with strategy.scope():
    model= model()
    model.compile(optimizer = Adam(learning_rate= 0.001), loss='mse', metrics = ['acc'])

lr_reduction = ReduceLROnPlateau(monitor= 'loss', patience= 3, verbose= 1, factor= 0.5, min_lr= 0.0001)
model.fit(X_train, Y_train, validation_split=0.3, epochs= 40, batch_size=32, callbacks=[lr_reduction], verbose= 1)

*Uncomment the following line to save your model*

In [ ]:
#model.save('colorizer.model')

**IV. Colorize!**

In [ ]:
def colorize(path):
    img= img_to_array(load_img(path))/255   # Standardize RGB image array for the RGB-LAB transformation
    img = resize(img ,(256,256))    # (256, 256, 3) standard shape for all images

    # extract l layer from lab
    lab_img= rgb2lab(img)
    l_img = lab_img[:,:,:1]
    assert l_img.shape == (256,256,1) , "L channel matrix shape is wrong"
    
    input0= l_img.reshape((1,)+ l_img.shape)

    # generate gray image 
    gray_img= rgb2gray(img)
    gray_rgb_img= gray2rgb(gray_img)

    input1= get_inception_embedding(gray_rgb_img)
    input1= input1.reshape((1,)+ input1.shape)
    assert input1.shape == (1, 1000) , "input 2 shape is wrong"

    input= [input0, input1]
    ab_predict= model.predict(input)[0]*128
    assert ab_predict.shape == (256,256,2) , "AB channels matrices shape is wrong"

    result_lab= np.zeros(img.shape)
    result_lab[:,:,:1], result_lab[:,:,1:]= l_img, ab_predict    # Give backthe original values range for AB
    result_rgb= lab2rgb(result_lab)

    return array_to_img(result_rgb)

*Give the path of your image*

In [ ]:
img_path= ''

In [ ]:
if img_path == '':
    print('Give a correct image path')
else:
    img= colorize(img_path)
    plt.imshow(img)